In [ ]:
# Implied Volatility Finder


In [ ]:
from libs import my_functions
import libs.universe as tickers
import pandas as pd
import pprint


In [ ]:
start = '2023-09-01'
end = '2023-09-07'
tickers_list = tickers.Universe


In [ ]:
result = my_functions.Alpaca_Markets.get_stock_data(tickers_list, '23Hour', start, end)
result


In [ ]:
iv_result = my_functions.calculate_volatility(result, tickers_list, '2023-06-01', end)
iv_resultsorted_iv = iv_result.sort_values(ascending = False).head(20)
iv_resultsorted_iv


In [ ]:
polygon_response = my_functions.Polygon_Markets.get_stock_data('AAPL', 1, 'day', '2023-09-01', '2023-09-12', 50000)
polygon_response


In [ ]:
polygon_options = my_functions.Polygon_Markets.get_options_data('F', '2023-09-18', '2023-10-30', 20, 150)


In [ ]:
opt = pprint.pprint(polygon_options)

In [ ]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint
import json

load_dotenv()

polygon_key = os.getenv("POLYGON_API_KEY")

url = 'https://api.polygon.io/v3/snapshot/options/'
url += 'EVRI'
url+= f'?apiKey={polygon_key}'

res = requests.get(url)
res_formatted = res.json()
pprint(res_formatted)


In [ ]:
exp_date = res_formatted['results'][0]['details']['expiration_date']
strike = res_formatted['results'][0]['details']['strike_price']
type = res_formatted['results'][0]['details']['contract_type']
print(f'Expiration Date: {exp_date}\nStrike: {strike}\nContract Type: {type}')


In [1]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint
import json
from libs import universe
import pandas as pd

load_dotenv()

key = os.getenv('TDAMERITRADE_DEV_KEY')
contract_type = 'PUT'
count = 1
range_values = 'OTM'
to_date = '2023-10-13'
expiration_month = 'OCT'

data = {}
my_data = []

for i in range(len(universe.Big_Guys)):

    url = 'https://api.tdameritrade.com/v1/marketdata/chains?'
    url += f'apikey={key}'
    url += f'&symbol={universe.Big_Guys[i]}'
    url += f'&contractType={contract_type}'
    url += f'&strikeCount={count}'
    url += f'&range={range_values}'
    url += f'&toDate={to_date}'
    url += f'&expMonth={expiration_month}'

    td_resp = requests.get(url)
    symbol_data = td_resp.content.decode('ASCII')
    res_again = json.loads(symbol_data)

    if res_again["status"] == 'SUCCESS':
        data["symbol"] = res_again['symbol']
        data["underlying price"] = res_again['underlyingPrice']
        data["strike price"] = float(list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0])
        data["volatility"] = res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]][list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0]][0]['volatility']
        data["theoretical volatility"] = res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]][list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0]][0]['theoreticalVolatility']

        my_data.append(data.copy())
        



In [2]:
data_df = pd.DataFrame(my_data).set_index('symbol')
data_df.sort_values(by=['strike price'], ascending=False, inplace=True)
data_df


,underlying price,strike price,volatility,theoretical volatility
symbol,,,,
VLO,126.75,127.0,32.707,29.0
XOM,107.43,108.0,24.724,29.0
CAH,89.02,90.0,17.291,29.0
SHEL,64.27,65.0,20.01,29.0
KO,53.12,54.0,20.734,29.0
AMC,9.32,9.5,NaN,29.0
